## Building a Perceptron

**0) Loading and Preparing Data**

Importing libraries:

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import datasets

<br>

Importing the iris dataset:

In [ ]:
iris  = datasets.load_iris()
names = iris.target_names

We want to start with two classes only: Setosa and "not Setosa"...

In [ ]:
mult  = [50,50,50]

Target = []
for n, m in zip(names, mult):
    Target.extend([n] * m)

indices = [i for i, x in enumerate(Target) if x == 'setosa']

Targetnum          = iris.target*0
Targetnum[indices] = 1

...and also with two features:

In [ ]:
X    = iris.data
X2D  = X[:,1:3]

In [ ]:
print(X2D)

In [ ]:
print(X)

In [ ]:
print(Target)

In [ ]:
print(Targetnum)

<br>

**1) Building the Neuron/Perceptron**

In [ ]:
import numpy as np

def Neuron1(Data, Target, alpha = 0.01):
    
#Data   :          the input matrix
#Target :          the target vector t for E = 0.5*(t - y)**2
#alpha  :          learning rate    

    if len(Target.shape) == 1:
        Target = Target.reshape(len(Target),1)

    [rows, cols] = Data.shape
 
    N = rows #N: Number of observations
    I = cols #I: Number of input channels

    #initializing weights incl bias:
    W     = np.random.normal(-1,1,(I + 1,1))

    #adding bias column (see slides)
    bias  = np.ones((N,1))
    Input = np.hstack((Data, bias))

    #calculating prediction
    net   = np.dot(Input, W)    #net output
    Y     = 1/(1 + np.exp(-net))#activation here: sigmoid

    #target vs output
    Error     = Target - Y
    
    #backpropagation
    dY     = Y*(1 - Y)
    dE     = Error
    
    W[:-1] = W[:-1] - alpha*np.dot(-Input[:,:-1].transpose(), dY*dE) #for weights
    W[-1]  = W[-1]  - alpha*np.dot(-dY.transpose(), dE)              #for bias

    return W

<br>

Let us run the neuron for X and X2D and check the weights

In [ ]:
W = Neuron1(X2D,Targetnum)
print(W) 

In [ ]:
W = Neuron1(X,Targetnum)
print(W) 

<br>

Now we want to see, if the weights converge and the error reduces if we run the code for many iterations. First we need to set the number of iterations as an input argument and then run a *for* loop within the neuron. Next, we add a plotting part at the end of the function to illustrate the changes of the weights, the bias and the error, hence the learning process.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def Neuron2(Niter, Data, Target, alpha = 0.01):

#Niter  :          number of iterations    
#Data   :          the input matrix
#Target :          the target vector t for E = 0.5*(t - y)**2
#alpha  :          learning rate   

    if len(Target.shape) == 1:
        Target = Target.reshape(len(Target),1)
        
    [rows, cols] = Data.shape
 
    N = rows #N: Number of observations
    I = cols #I: Number of input channels

    #initializing weights incl bias:
    W     = np.random.normal(-1,1,(I + 1,1))

    #adding bias column (see slides)
    bias  = np.ones((N,1))
    Input = np.hstack((Data, bias))

    MSE   = np.zeros((Niter,1))  #monitoring E
    Wei   = np.zeros((Niter,I + 1))#monitoring weights and bias

    for n in range(Niter):

        #calculating prediction
        net   = np.dot(Input, W)    #net output
        Y     = 1/(1 + np.exp(-net))#activation here: sigmoid
    
        #target vs output
        Error     = Target - Y
        
        #backpropagation
        dY     = Y*(1 - Y)
        dE     = Error
        
        W[:-1] = W[:-1] - alpha*np.dot(-Input[:,:-1].transpose(), dY*dE) #for weights
        W[-1]  = W[-1]  - alpha*np.dot(-dY.transpose(), dE)              #for bias

        #saving results    
        MSE[n]   = sum(Error**2)/N
        Wei[n,:] = W.transpose()

    
########plotting#################################################################

    fig, axs = plt.subplots(2, 1)
    axs[0].plot(range(Niter), MSE, c = 'black', linewidth = 3)
    axs[0].set_title('MSE')
    axs[0].set_xlabel('iteration')
    axs[0].set_yscale('log')
    axs[1].plot(range(Niter), Wei, linewidth = 3)
    axs[1].set_title('weights')
    axs[1].set_xlabel('iteration')
    fig.tight_layout(pad = 1.0)
    plt.show()


    return Wei

In [ ]:
Wei = Neuron2(5000, X2D, Targetnum) 

In [ ]:
Wei = Neuron2(5000, X, Targetnum)

We see that for most runs, the error reduces and the weights converge. But in some cases the neuron doesn't seem to find a good minimum. This is because we neither have implemented an adaptive learning rate, nor a momentum or any regularization.

<br>

We still want to understand the learning process in more detail. Run the function NeuronContour.py for different numbers of iterations and explore how the threshold emerges.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def NeuronContour(Niter, Data, Target, alpha = 0.01):

#Niter  :          number of iterations    
#Data   :          the input matrix
#Target :          the target vector t for E = 0.5*(t - y)**2
#alpha  :          learning rate

    #creating meshgrid through the data range for illustration
    Ngrid = 100

    x1 = np.linspace(Data[:,0].min(), Data[:,0].max(), Ngrid)
    x2 = np.linspace(Data[:,1].min(), Data[:,1].max(), Ngrid)
    
    [xx1, xx2] = np.meshgrid(x1, x2)


    if len(Target.shape) == 1:
        Target = Target.reshape(len(Target),1)
        
    [rows, cols] = Data.shape
 
    N = rows #N: Number of observations
    I = cols #I: Number of input channels

    #initializing weights incl bias:
    W     = np.random.normal(-1,1,(I + 1,1))

    #adding bias column (see slides)
    bias  = np.ones((N,1))
    Input = np.hstack((Data, bias))


    for n in range(Niter):

        #calculating prediction
        net   = np.dot(Input, W)    #net output
        Y     = 1/(1 + np.exp(-net))#activation here: sigmoid
    
        #target vs output
        Error     = Target - Y
        
        #backpropagation
        dY     = Y*(1 - Y)
        dE     = Error
        
        W[:-1] = W[:-1] - alpha*np.dot(-Input[:,:-1].transpose(), dY*dE) #for weights
        W[-1]  = W[-1]  - alpha*np.dot(-dY.transpose(), dE)              #for bias



    
########plotting#################################################################
    XY        = np.c_[xx1.ravel(), xx2.ravel()]
    L         = XY.shape[0]
    Inputcont = np.hstack((XY, np.ones((L,1))))
    netcont   = np.dot(Inputcont, W)
    Ycont     = 1/(1 + np.exp(-netcont))
    Ycont     = Ycont[:,0:2].reshape(Ngrid,Ngrid)
    Y         = np.round(Y)
    
    IdxTrue0 = np.argwhere(Target == 0)
    IdxTrue1 = np.argwhere(Target == 1)
    
    IdxPred0 = np.argwhere(Y == 0)
    IdxPred1 = np.argwhere(Y == 1)

    CM = "Blues"
    
    plt.contourf(xx1, xx2, Ycont, cmap = CM, alpha = 0.3, levels = 100)
    plt.scatter(Data[IdxTrue0, 0], Data[IdxTrue0, 1], color = [255/256,69/256,0], marker = '.')
    plt.scatter(Data[IdxTrue1, 0], Data[IdxTrue1, 1], color = [46/256,139/256,87/256], marker = '.')
    plt.scatter(Data[IdxPred0, 0], Data[IdxPred0, 1], edgecolors = [255/256,69/256,0], marker = 'o', facecolors = 'none')
    plt.scatter(Data[IdxPred1, 0], Data[IdxPred1, 1], edgecolors = [46/256,139/256,87/256], marker = 'o', facecolors = 'none')


<br>

In [ ]:
NeuronContour(3, X2D, Targetnum)

In [ ]:
NeuronContour(5, X2D, Targetnum)

In [ ]:
NeuronContour(100, X2D, Targetnum)

In [ ]:
NeuronContour(10000, X2D, Targetnum)

<br>